In [1]:

import bs4
from bs4 import BeautifulSoup
import lxml

with open("data/city.xml") as fp:
    soup = BeautifulSoup(fp, "xml")

In [2]:
def find_tag_children(node):
    tags=list()
    for child in node.children:
        if type(child)==bs4.element.Tag:
            tags.append(child)
    return tags

In [3]:
children=find_tag_children(soup.contents[0])
print(len(children))
for i in children:
    print(type(i))

3
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [4]:
from collections import namedtuple

ValueT=namedtuple('ValueT',['cname','old_code','sng_code','gb_code','gb_code_path','status'])

def dfs(node:bs4.element.Tag,paths:list,values:list):
    cname=node.get('cname')
    old_code=node.get('name')
    sng_code=node.get('deprecated_code')
    gb_code=node.get('gb_code')

    gb_code_path=';'.join(paths) if len(paths)!=0 else None
    status=0 if node.get('status')=='active' else 1

    value_t=ValueT(cname,old_code,sng_code,gb_code,gb_code_path,status)
    if gb_code is not None and cname is not None:
        values.append(value_t)
        print(int(gb_code))

    paths.append(gb_code)
    children=find_tag_children(node)
    for child in children:
        dfs(child,paths,values)
    paths.pop()
root =soup.contents[0]
paths=[]
values=[]
provinces=find_tag_children(root)
for province in provinces:
    dfs(province,paths,values)



156
1156
710000
719900
710200
712200
710700
711500
712300
712000
710100
712100
710500
710400
711100
710600
710900
711700
711400
711300
711900
713100
713200
713300
540000
549900
542100
542121
542122
542123
542124
542125
542126
542127
542128
542129
542132
542133
540100
540102
540121
540122
540123
540124
540125
540126
540127
542600
542621
542622
542623
542624
542625
542626
542627
540200
540202
540221
540222
540223
540224
540225
540226
540227
540228
540229
540230
540231
540232
540233
540234
540235
540236
540237
542200
542221
542222
542223
542224
542225
542226
542227
542228
542229
542231
542232
542233
542500
542521
542522
542523
542524
542525
542526
542527
542400
542421
542422
542423
542424
542425
542426
542427
542428
542429
542430
542431
630000
639900
632200
632221
632222
632223
632224
630200
630202
630221
630222
630223
630224
630225
632500
632521
632522
632523
632524
632525
632800
632801
632802
632821
632822
632823
632824
632825
632826
630100
630102
630103
630104
630105
630121
630122
6301

In [5]:
from typing import List
def make_sql(values:List[ValueT]):
    sql_template="""
use ndmp;
insert into rulelab_hierarchical_area
(cname,old_code,sng_code,gb_code,gb_code_path,status)
values {}
    """
    sql_values=[]
    for value_t in values:
        cname=f"'{value_t.cname}'"
        old_code=f"{value_t.old_code}" if value_t.old_code is not None else 'null'
        sng_code=f"{value_t.sng_code}" if value_t.sng_code is not None else 'null'
        gb_code=f"{value_t.gb_code}"
        gb_code_path=f"'{value_t.gb_code_path}'" if value_t.gb_code_path is not None else 'null'
        status=f"{value_t.status}"

        sql_values.append("\n("+",".join([cname,old_code,sng_code,gb_code,gb_code_path,status])+")")
    return sql_template.strip().format(",".join(sql_values))



sql=make_sql(values)


In [6]:
with open('data/insert.sql','w') as writer:
    writer.writelines(sql)

In [81]:
values[0]

ValueT(cname='中国未知', old_code='980000', sng_code='362', gb_code='0156', gb_code_path=None, status=0)